In [1]:
import os
import re
import requests
from datetime import datetime

from bs4 import BeautifulSoup, NavigableString

import vertexai
from vertexai.generative_models import (
    GenerativeModel,
    GenerationConfig,
    Tool,
    FunctionDeclaration,
    Part)

In [2]:
# initialise vertexai
project_id = !gcloud config get project
project_id = project_id[0]

vertexai.init(project=project_id, location="us-central1")

In [3]:
!gcloud config get project

qwiklabs-gcp-00-b65ff4dbd84e


In [4]:
project_id

'qwiklabs-gcp-00-b65ff4dbd84e'

# Task 2. Download SEC filings and parse text to analyze


## Boston Scientific

In [6]:
# IMPORTANT! You must set these variables with REAL VALUES for the correct headers to be in place to access the API
PARTNER_COMPANY = "Boston Scientific" # "YOUR COMPANY'S NAME"
PARTNER_WEBSITE = "https://www.bostonscientific.com/en-US/home.html" # "https://your-companys-website"

headers = {"User-Agent": f"{PARTNER_COMPANY} +{PARTNER_WEBSITE}"}

In [7]:
headers

{'User-Agent': 'Boston Scientific +https://www.bostonscientific.com/en-US/home.html'}

In [8]:
# 2 fns
# download_single_filing - It downloads a single SEC filing identified by a company's Central Index Key (CIK), the form type(an annual 10-K or quarterly 10-Q) and the date of filing.
# download_range_of_filings - It queries all the filings of a company to find filings within a date range of a particular form type and download them.

BASE_DIR = "filings" # Directory for storing raw filings

def download_single_filing(url, cik, form, date, file_extension):
    """
    This function downloads and saves a SEC filing from a specified URL.

    Args:
        cik (str): Central Index Key (CIK) for the company.
        form (str): The type of SEC filing (e.g., 10-K, 10-Q).
        date (str): The filing date in YYYY-MM-DD format.
        file_extension (str): File extension for saved file (usually 'txt' or 'zip').
    """

    # Define request headers to simulate a browser visit
    response = requests.get(url, headers=headers)

    if response.status_code == 200:

        # Make the directory accord
        dir_name = f"{BASE_DIR}/{cik}"
        os.makedirs(dir_name, exist_ok=True)
        file_path = f"{dir_name}/{form}_{date}.{file_extension}"

        with open(file_path, 'wb') as file:
            file.write(response.content)

        print(f"Downloaded {form} for CIK {cik} on {date} to {file_path}")

        return file_path

    else:
        print(f"Failed to download {form} for CIK {cik} on {date}. Status code: {response.status_code}")

        return None

    
    
    
def download_range_of_filings(cik, starting_year_and_quarter,
                            ending_year_and_quarter, include_10q = False):
    """
    Download filings from EDGAR for a given CIK and clean them up.

    Args:
        cik (str): Central Index Key (CIK)
        starting_year_and_quarter (str): Specified in the format "2023 Q1"
        ending_year_and_quarter (str): Specified in the format "2024 Q4"
        include_10q (bool): Whether to include 10-Qs in addition to 10-Ks
    """

    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    headers = {'User-Agent': 'Google Partner Learning Services Demo +https://partners.cloud.google.com/learn'}
    response = requests.get(url, headers=headers)

    forms_to_download = {"10-K", "10-Q"} if include_10q else {"10-K"}

    start_year, start_quarter = starting_year_and_quarter.split()
    end_year, end_quarter = ending_year_and_quarter.split()

    if response.status_code == 200:
        data = response.json()

        filing_paths = []
        for filing_date, form, accession_number in zip(data['filings']['recent']['filingDate'], data['filings']['recent']['form'], data['filings']['recent']['accessionNumber']):
            if (form in forms_to_download) and (int(start_year) <= datetime.strptime(filing_date, '%Y-%m-%d').year <= int(end_year)):
                file_path = download_single_filing(
                    f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number.replace('-', '')}/{accession_number}.txt",
                    cik, form, filing_date, 'htm'
                )
                filing_paths.append (file_path)
        return filing_paths
    else:
        print("Error from call.")
        print(response)
        
        

In [ ]:
# # https://www.sec.gov/cgi-bin/cik_lookup
	
# Results of EDGAR CIK Lookup
# Your search had 1 hit.

# CIK Code     Company Name
# 0000885725   BOSTON SCIENTIFIC CORP

# Generated at 03:15:01 EDT on October 13, 2025



In [9]:
bos_cik = "0000885725"

download_range_of_filings(cik=bos_cik,
                starting_year_and_quarter="2024 Q1",
                ending_year_and_quarter="2024 Q4")

Downloaded 10-K for CIK 0000885725 on 2024-02-20 to filings/0000885725/10-K_2024-02-20.htm


['filings/0000885725/10-K_2024-02-20.htm']

In [10]:
model = GenerativeModel("gemini-2.0-flash",
                        generation_config=GenerationConfig(temperature=0),
                       )

/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [11]:
# downloaded_path = "filings/0001652044/10-K_2024-01-31.htm"
downloaded_path = "filings/0000885725/10-K_2024-02-20.htm"

with open(downloaded_path, 'r') as f:
    filing_text = f.read()

response = model.count_tokens(filing_text)
print(response)    

total_tokens: 6695807
total_billable_characters: 15866556
prompt_tokens_details {
  modality: TEXT
  token_count: 6695807
}



In [21]:
# SEC filings structure/headings
items = ['Business',
        'Risk Factors',
        'Unresolved Staff Comments',
        'Properties', 'Legal Proceedings',
        'Mine Safety Disclosures',
        'Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities',
        'Management’s Discussion and Analysis of Financial Condition and Results of Operations',
        'Quantitative and Qualitative Disclosures About Market Risk',
        'Financial Statements and Supplementary Data',
        'Changes in and Disagreements with Accountants on Accounting and Financial Disclosure',
        'Controls and Procedures',
        'Other Information',
        'Disclosure Regarding Foreign Jurisdictions that Prevent Inspections',
        'Directors, Executive Officers, and Corporate Governance',
        'Executive Compensation',
        'Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters',
        'Certain Relationships and Related Transactions, and Director Independence',
        'Principal Accountant Fees and Services',
        'Exhibit and Financial Statement Schedules',
        'Form 10-K Summary']

def find_div_id(soup, item_name: str):
    # BOSTON SCIENTIFIC USES UPPER CASE SECTION HEADERS
    found_tag = soup.find('a', string=item_name.upper())
    if found_tag:
        return found_tag["href"].strip('#')
    else:
        print(f"Couldn't find a matching tag for: {item_name}")
        return None

def get_text_between(soup, cur_name, end_name):
    cur = soup.find('div', id=find_div_id(soup, cur_name)).next_sibling
    end = soup.find('div', id=find_div_id(soup, end_name))
    while cur and cur != end:
        if isinstance(cur, NavigableString):
            text = cur.strip()
            if len(text):
                yield text
        cur = cur.next_element

def get_items_from_filings(item_names, filing_paths):

    print("Items of interest: " + ", ".join(item_names) + "\n")
    item_strings = {item: f"<{item}>\n" for item in item_names}

    for path in filing_paths:
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read()

        soup = BeautifulSoup(content, 'html.parser')

        for item in item_names:
            item_index = items.index(item)
            item_index = item_index if item_index < len(items) - 1 else 0
            item_output = ' '.join(text for text in get_text_between(soup, item, items[item_index + 1]))
            item_strings[item] += f"From {os.path.basename(path)}" + "\n" + item_output + "\n"

    return "\n\n".join(item_strings.values())    

In [20]:
item_names = ["Management’s Discussion and Analysis of Financial Condition and Results of Operations"]

# boston scientific
downloaded_path = "filings/0000885725/10-K_2024-02-20.htm"
report = get_items_from_filings(item_names, [downloaded_path])

# Print the first 2,000 characters as an example.
print(report[0:2000] + "...")

Items of interest: Management’s Discussion and Analysis of Financial Condition and Results of Operations

<Management’s Discussion and Analysis of Financial Condition and Results of Operations>
From 10-K_2024-02-20.htm
ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS The following discussion and analysis provides information management believes to be relevant to understanding the financial condition and results of operations of Boston Scientific Corporation and its subsidiaries for the years ended December 31, 2023 and 2022. For a full understanding of our financial condition and results of operations, this discussion should be read in conjunction with our consolidated financial statements and accompanying notes included in Item 8. Financial Statements and Supplementary Data of this Annual Report on Form 10-K. For additional information on our financial condition and results of operations for the year ended December 31, 2021, refer to Item 7

In [14]:
response = model.count_tokens(report)
print(response)

total_tokens: 4535809
total_billable_characters: 10565142
prompt_tokens_details {
  modality: TEXT
  token_count: 4535809
}



## Alphabet/Google

In [15]:
alphabet_cik = "0001652044"

download_range_of_filings(cik=alphabet_cik,
                starting_year_and_quarter="2024 Q1",
                ending_year_and_quarter="2024 Q4")

Downloaded 10-K for CIK 0001652044 on 2024-01-31 to filings/0001652044/10-K_2024-01-31.htm


['filings/0001652044/10-K_2024-01-31.htm']

In [16]:
model = GenerativeModel("gemini-2.0-flash",
                        generation_config=GenerationConfig(temperature=0),
                       )

downloaded_path = "filings/0001652044/10-K_2024-01-31.htm"

with open(downloaded_path, 'r') as f:
    filing_text = f.read()

response = model.count_tokens(filing_text)
print(response)    

total_tokens: 5798629
total_billable_characters: 13029439
prompt_tokens_details {
  modality: TEXT
  token_count: 5798629
}



In [22]:

def find_div_id(soup, item_name: str):
    # BOSTON SCIENTIFIC USES UPPER CASE SECTION HEADERS
    # found_tag = soup.find('a', string=item_name.upper())
    # ALPHABET/GOOGLE USES Proper/Title Case 
    found_tag = soup.find('a', string=item_name)
    if found_tag:
        return found_tag["href"].strip('#')
    else:
        print(f"Couldn't find a matching tag for: {item_name}")
        return None
    
    
    

item_names = ["Management’s Discussion and Analysis of Financial Condition and Results of Operations"]

# google/alphabet
downloaded_path = "filings/0001652044/10-K_2024-01-31.htm"
report = get_items_from_filings(item_names, [downloaded_path])

# Print the first 2,000 characters as an example.
print(report[0:2000] + "...")

Items of interest: Management’s Discussion and Analysis of Financial Condition and Results of Operations

<Management’s Discussion and Analysis of Financial Condition and Results of Operations>
From 10-K_2024-01-31.htm
Table of Contents Alphabet Inc. ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS Please read the following discussion and analysis of our financial condition and results of operations together with “Note about Forward-Looking Statements,” Part I, Item 1 "Business," Part I, Item 1A "Risk Factors," and our consolidated financial statements and related notes included under Item 8 of this Annual Report on Form 10-K. The following section generally discusses 2023 results compared to 2022 results. Discussion of 2022 results compared to 2021 results to the extent not included in this report can be found in Item 7 of our 2022 Annual Report on Form 10-K. Understanding Alphabet’s Financial Results Alphabet is a collection of businesses 

In [18]:
response = model.count_tokens(report)
print(response)

total_tokens: 12688
total_billable_characters: 50620
prompt_tokens_details {
  modality: TEXT
  token_count: 12688
}



# Task 3. Let Gemini look up companies' CIKs via Google Search


## NO GROUNDING

In [23]:
#  The CIK of Summit Therapeutics is actually 0001599298, but without the ability to look it up, Gemini provides some alternative numbers.
model.generate_content("What is Summit Therapeutics' CIK with the SEC?").text

"Summit Therapeutics Inc.'s CIK (Central Index Key) with the SEC is **0001349558**.\n"

## WITH GROUNDING

In [24]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch, HttpOptions
from IPython.display import display, HTML
import os

project_id = os.environ['GOOGLE_CLOUD_PROJECT'] = 'qwiklabs-gcp-00-b65ff4dbd84e'
location = os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

# Initialize the client, explicitly passing project and location for Vertex AI
client = genai.Client(project=project_id, location=location, http_options=HttpOptions(api_version="v1"))
model_id = "gemini-2.0-flash"

# Configure Google Search as a tool for grounding
search_tool = Tool(
    google_search=GoogleSearch()
)

# Helper function to lookup a company's SEC CIK
def lookup_cik(company_name: str) -> str:
    prompt = f"""
Find the Securities and Exchange Commission's Central Index Key (CIK)
for {company_name}.
Return only the 10-digit integer CIK including leading zeroes.
"""
    # Generate content with grounding via Google Search
    response = client.models.generate_content(
        model=model_id,
        contents=prompt,
        config=GenerateContentConfig(
            tools=[search_tool],
            response_modalities=["TEXT"],
        )
    )

    # Concatenate all text parts of the response
    cik = "".join(part.text for part in response.candidates[0].content.parts).strip()
    print(f"Lookup for CIK of {company_name} resulted in: {cik}\n")

    # Display the grounding metadata (search suggestions and sources)
    if hasattr(response.candidates[0], 'grounding_metadata'):
        display(
            HTML(
                response.candidates[0]
                .grounding_metadata.search_entry_point.rendered_content
            )
        )

    return cik

In [25]:
lookup_cik("Summit Therapeutics")

Lookup for CIK of Summit Therapeutics resulted in: The Securities and Exchange Commission (SEC) Central Index Key (CIK) for Summit Therapeutics is 0001599298.



'The Securities and Exchange Commission (SEC) Central Index Key (CIK) for Summit Therapeutics is 0001599298.'

In [26]:
lookup_cik("BOSTON SCIENTIFIC")

Lookup for CIK of BOSTON SCIENTIFIC resulted in: The Securities and Exchange Commission (SEC) Central Index Key (CIK) for BOSTON SCIENTIFIC is 0000885725.



'The Securities and Exchange Commission (SEC) Central Index Key (CIK) for BOSTON SCIENTIFIC is 0000885725.'

In [27]:
# FunctionDeclaration TO lookup_cik FN DEFINED EARLIER
lookup_cik_fd = FunctionDeclaration(
    name="lookup_cik",
    description="Look up a company's CIK used for its SEC filings.",
    parameters={
        "type": "object",
        "properties": {
            "company_name": {
                "type": "string",
                "description": "The name of the company to look up."
            },
        },
        "required": [
            "company_name"
        ]
    },
)

# Note: The search_tool loaded above cannot be combined with other Tools when passed to Gemini, so in order to create a Tool that combines it and other functions, you can create a dedicated model instance and invoke that via another function as you are doing here.


# Task 4. Empower Gemini to retrieve document sections from relevant years


In [28]:
retrieve_filings_fd = FunctionDeclaration(
    name="retrieve_filings",
    description="Retrieve filings from the SEC EDGAR API for a company within a date range.",
    parameters={
        "type": "object",
        "properties": {
            "cik": {
                "type": "string",
                "description": "The CIK of a company whose documents will be retrieved"
            },
            "starting_year_and_quarter": {
                "type": "string",
                "description": "The first report quarter year and quarter in the format: 2024 Q1"
            },
            "ending_year_and_quarter": {
                "type": "string",
                "description": "The year and quarter in the format: 2024 Q1"
            },
            "include_quarterly_reports": {
                "type": "boolean",
                "description": "Whether to include 10-Q quarterly filings in addition to annual 10-K filings"
            },
            "items_of_interest": {
                "description": "An array of one or more section (called items) of interest from 10-K or 10-Q filings",
                "type": "array",
                "items": {
                "type": "string",
                "enum": [
                    "Business",
                    "Risk Factors",
                    "Unresolved Staff Comments",
                    "Properties",
                    "Legal Proceedings",
                    "Mine Safety Disclosures",
                    "Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities",
                    "Management’s Discussion and Analysis of Financial Condition and Results of Operations",
                    "Quantitative and Qualitative Disclosures About Market Risk",
                    "Financial Statements and Supplementary Data",
                    "Changes in and Disagreements with Accountants on Accounting and Financial Disclosure",
                    "Controls and Procedures",
                    "Other Information",
                    "Disclosure Regarding Foreign Jurisdictions that Prevent Inspections",
                    "Directors, Executive Officers and Corporate Governance",
                    "Executive Compensation",
                    "Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters",
                    "Certain Relationships and Related Transactions, and Director Independence",
                    "Principal Accountant Fees and Services",
                    "Exhibit and Financial Statement Schedules",
                    "Form 10-K Summary"
                ]
                }
        }
        },
        "required": [
            "cik",
            "starting_year_and_quarter",
            "ending_year_and_quarter",
            "items_of_interest"
        ]
    },
)    

In [29]:
# fd = fn-declaration
sec_tool = Tool(function_declarations=[retrieve_filings_fd,
                                    lookup_cik_fd])    

In [30]:
response1 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Find the SEC CIK for Acme Corp. Return only the 10-digit CIK.",
    config=GenerateContentConfig(tools=[search_tool], response_modalities=["TEXT"])
)
cik = "".join(p.text for p in response1.candidates[0].content.parts).strip()





from vertexai.generative_models import GenerativeModel, GenerationConfig, Tool, FunctionDeclaration

# Define your function declarations (lookup_cik_fd, retrieve_filings_fd)
sec_tool = Tool(function_declarations=[lookup_cik_fd, retrieve_filings_fd])

system_instruction = """
    - You are a research assistant to a financial analyst.
    - Answer the user's question with an analysis, basing your response on information
    used in filings from the SEC EDGAR database.
    - Quote the SEC filing documents to support your analysis.
    - If you are not certain about a CIK, use your tool to look it up.
    - The current date is {current_date}.
    """.format(
        current_date=datetime.today().strftime("%Y-%m-%d")
    )

model = GenerativeModel(
    model_name="gemini-2.0-flash",
    generation_config=GenerationConfig(temperature=0),
    system_instruction=system_instruction,
    tools=[sec_tool]
)

# 'cik' here uses the information from 'response1'
response2 = model.generate_content(
    contents=f"Retrieve the 10-K for CIK {cik} covering 2023 Q4, and extract 'Risk Factors'."
)

In [31]:
response1

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text="""I found two companies with the name "Acme" in their name.

*   **ACME UNITED CORP:** The SEC CIK is 0000002098.
*   **ACME ELECTRIC CORP:** The SEC CIK is 0000002070.

"""
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      grounding_metadata=GroundingMetadata(
        grounding_chunks=[
          GroundingChunk(
            web=GroundingChunkWeb(
              domain=<... Max depth ...>,
              title=<... Max depth ...>,
              uri=<... Max depth ...>
            )
          ),
          GroundingChunk(
            web=GroundingChunkWeb(
              domain=<... Max depth ...>,
              title=<... Max depth ...>,
              uri=<... Max depth ...>
            )
          ),
        ],
        grounding_supports=[
          GroundingSuppor

In [32]:
cik

'I found two companies with the name "Acme" in their name.\n\n*   **ACME UNITED CORP:** The SEC CIK is 0000002098.\n*   **ACME ELECTRIC CORP:** The SEC CIK is 0000002070.'

In [33]:
response2

candidates {
  content {
    role: "model"
    parts {
      function_call {
        name: "retrieve_filings"
        args {
          fields {
            key: "starting_year_and_quarter"
            value {
              string_value: "2023 Q4"
            }
          }
          fields {
            key: "items_of_interest"
            value {
              list_value {
                values {
                  string_value: "Risk Factors"
                }
              }
            }
          }
          fields {
            key: "ending_year_and_quarter"
            value {
              string_value: "2023 Q4"
            }
          }
          fields {
            key: "cik"
            value {
              string_value: "0000002098"
            }
          }
        }
      }
    }
    parts {
      function_call {
        name: "retrieve_filings"
        args {
          fields {
            key: "starting_year_and_quarter"
            value {
              string_value:

When you ask Gemini a question related to a public company, it may return a response, or it may return a request for a function call. Starting with Gemini 2.0, it may ask for function calls in separate rounds of chat, or multiple parallel function calls in a single round of response. If multiple function calls are requested, your response must include a function response for each function call Gemini has requested (the Part.from_function_response section). To handle these cases, it is usually very helpful to define a handle_response function:

In [34]:
def handle_response(response):

    parts_for_inner_response = []
    for part in response.candidates[0].content.parts:
        # If the content part has an attribute called 'text'
        if hasattr(part, "text"):
            print("\n" + part.text)
        # If the content has an attribute called 'function_call'
        if part.function_call:
            function_call = part.function_call
            try:
                if function_call.name == "lookup_cik":
                    cik = lookup_cik(function_call.args["company_name"])
                    parts_for_inner_response.append(
                        Part.from_function_response(
                            name="lookup_cik",
                            response={
                                "content": cik,
                            },
                        )
                    )
                if function_call.name == "retrieve_filings":
                    context = ""
                    filing_paths = download_range_of_filings(function_call.args["cik"],
                                    function_call.args["starting_year_and_quarter"],
                                    function_call.args["ending_year_and_quarter"],
                                    function_call.args.get("include_10q", False)
                    )
                    if filing_paths:
                        # TODO: Load cleaned filing docs
                        report = get_items_from_filings(function_call.args["items_of_interest"], filing_paths)
                        parts_for_inner_response.append(
                            Part.from_function_response(
                                name="retrieve_filings",
                                response={
                                    "content": report,
                                },
                            )
                        )
                    else:
                        print("No valid filings found or an error was encountered in retrieving them.")
            except AttributeError as e:
                print("Exception:")
                print(response)
                print(part)
                print(e)
    if parts_for_inner_response:
        inner_response = chat.send_message(parts_for_inner_response)
        handle_response(inner_response)   

## Alphabet/GOOG vs AMZN risks

In [35]:
chat = model.start_chat()

In [36]:
response = chat.send_message("How do Alphabet's risks in 2024 compare to Amazon's?")
handle_response(response)

Lookup for CIK of Alphabet resulted in: The Securities and Exchange Commission's (SEC) Central Index Key (CIK) for Alphabet Inc. is 0001652044.



Lookup for CIK of Amazon resulted in: The Securities and Exchange Commission (SEC) Central Index Key (CIK) for Amazon is 0001018724.



Downloaded 10-K for CIK 0001652044 on 2024-01-31 to filings/0001652044/10-K_2024-01-31.htm
Items of interest: Risk Factors

Downloaded 10-K for CIK 0001018724 on 2024-02-02 to filings/0001018724/10-K_2024-02-02.htm
Items of interest: Risk Factors


Based on the 2024 SEC filings, here's a comparison of the risks faced by Alphabet and Amazon:

**Alphabet Inc. (GOOGL):**

*   **Competition:** Alphabet faces intense competition across various industries, including online advertising, AI technology, and devices. They must continue to innovate and provide useful products and services to remain competitive.
    > *"We face intense competition. If we do not continue to innovate and provide products and services that are useful to users, customers, and other partners, we may not remain competitive, which could harm our business, financial condition, and operating results."*
*   **AI Investment:** Expanding investment in AI is risky and requires significant resources.
    > *"We are expanding ou

## Home Depot past 3 years biz trend

In [37]:
chat = model.start_chat()
response = chat.send_message("How has Home Depot changed the way it describes its business over the past 3 years?")
handle_response(response)

Lookup for CIK of Home Depot resulted in: The Securities and Exchange Commission (SEC) Central Index Key (CIK) for Home Depot is 0000354950.



Downloaded 10-K for CIK 0000354950 on 2025-03-21 to filings/0000354950/10-K_2025-03-21.htm
Downloaded 10-K for CIK 0000354950 on 2024-03-13 to filings/0000354950/10-K_2024-03-13.htm
Downloaded 10-K for CIK 0000354950 on 2023-03-15 to filings/0000354950/10-K_2023-03-15.htm
Downloaded 10-K for CIK 0000354950 on 2022-03-23 to filings/0000354950/10-K_2022-03-23.htm
Items of interest: Business


Over the past three years, Home Depot has refined its description of its business, with a consistent emphasis on interconnected shopping experiences and strategic investments. Here's a breakdown of the key changes:

*   **2022 (Fiscal Year 2021):** Home Depot emphasized its interconnected, frictionless shopping experience, and highlighted its ability to operate successfully due to strategic investments. It focused on providing the best customer experience, being the low-cost provider, and being the most efficient investor of capital. The company served two primary customer groups: DIY and profession

# Revisit ACME again - 2 different CIKs

In [39]:
chat = model.start_chat()
response = chat.send_message("Retrieve CIKs for ACME Corp. Then retrieve the 10-K covering 2023 Q4, and extract 'Risk Factors'")
handle_response(response)

Lookup for CIK of ACME Corp resulted in: While there may be many companies with similar names, here are a few examples of how to find the CIK for different ACME companies:

*   **ACME Electric Corp:** The CIK is 0000002070.
*   **ACME United Corp:** The CIK is 0000002098.
*   **ACME, LLC:** The CIK is 1803200.

You can use the SEC's EDGAR system to find the CIK of any company that files with the SEC. To do so, go to the SEC's CIK Lookup tool and enter the company's name. The search results will provide a list of entities matching your search criteria and their CIK numbers.




There are multiple companies with similar names. Which CIK would you like to use?



In [41]:
chat = model.start_chat()
response = chat.send_message("Retrieve CIKs for ACME in Tom and Jerry. Then retrieve the 10-K covering 2023 Q4, and extract 'Risk Factors'")
handle_response(response)

Lookup for CIK of ACME in Tom and Jerry resulted in: I was unable to find the Securities and Exchange Commission's Central Index Key (CIK) for ACME in "Tom and Jerry." While the SEC assigns a unique 10-digit CIK number to entities and individuals that submit filings, there is no indication that the fictional ACME corporation from "Tom and Jerry" has any filings with the SEC that would warrant a CIK number.




I was unable to find the Securities and Exchange Commission's Central Index Key (CIK) for ACME in "Tom and Jerry." While the SEC assigns a unique 10-digit CIK number to entities and individuals that submit filings, there is no indication that the fictional ACME corporation from "Tom and Jerry" has any filings with the SEC that would warrant a CIK number.



In [44]:
chat = model.start_chat()
response = chat.send_message("Retrieve CIKs for ACME Electric Corp. Then retrieve the 10-K covering 2023 Q4, and summarise 'Risk Factors'")
handle_response(response)

Lookup for CIK of ACME Electric Corp. resulted in: The Securities and Exchange Commission's (SEC) Central Index Key (CIK) for ACME Electric Corp. is 0000002070.



No valid filings found or an error was encountered in retrieving them.


In [47]:
chat = model.start_chat()
response = chat.send_message("Retrieve CIKs for ACME United Corp. Then retrieve the 10-K covering 2023 Q4. Summarise the 10-K and extract the 'Risk Factors'")
handle_response(response)

Lookup for CIK of ACME United Corp resulted in: The Securities and Exchange Commission (SEC) Central Index Key (CIK) for ACME United Corp. is 0000002098.



Downloaded 10-K for CIK 0000002098 on 2023-03-10 to filings/0000002098/10-K_2023-03-10.htm
Items of interest: Risk Factors, Business

Couldn't find a matching tag for: Risk Factors
Couldn't find a matching tag for: Unresolved Staff Comments
Couldn't find a matching tag for: Business
Couldn't find a matching tag for: Risk Factors


InvalidArgument: 400 The input token count (2994790) exceeds the maximum number of tokens allowed (1048575).

In [46]:
chat = model.start_chat()
response = chat.send_message("Retrieve CIKs for ACME LLC. Then retrieve the 10-K covering 2023 Q4, and summarise 'Risk Factors'")
handle_response(response)

Lookup for CIK of ACME LLC resulted in: The Securities and Exchange Commission (SEC) Central Index Key (CIK) for ACME LLC is 0001803200.



No valid filings found or an error was encountered in retrieving them.
